This is the start 
For a simple comprehensive tutorial from gammapy, one can follow https://docs.gammapy.org/0.18.2/tutorials/spectrum_analysis.html

Here we break the post-DL3 analyses to simple and separate sections to ease the facilitation of all steps - 
# 1. Reduce DL3 data into Spectrum Dataset objects in OGIP files
# 2. Plot LC from the OGIP files
# 3. Plot SEDs from the OGIP files

# This example notebook, converts the provided DL3 files into Spectrum Dataset objects and saves the 1D counts spectra and Associated Response Function in OGIP format, as following:

## 1. Read the provided DL3 index files
## 2. Apply selection filters to the list of DL3 files
## 3. Define base geometry for the 1D spectrum
## 4. Generate some dataset makers for data reduction
## 5. Perform data reduction over all selected observations and compile them to a Dataset
## 6. Save the Dataset to OGIP files

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from regions import CircleSkyRegion
from pathlib import Path
import os

In [ ]:
from gammapy.data import DataStore

from gammapy.maps import Map, MapAxis, WcsNDMap, WcsGeom, RegionGeom
from gammapy.data import DataStore

from gammapy.datasets import (
    Datasets,
    SpectrumDataset,
    SpectrumDatasetOnOff,
)
from gammapy.makers import (
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)

from gammapy.visualization import plot_spectrum_datasets_off_regions

import astropy.units as u
from astropy.table import Table
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord, Angle

# 1. Parameters from user for selection of observations

In [ ]:
# Use the DL3 files produced for source dependent or independent analyses
base_dir = "../data/DL3/"
dir_path = base_dir + "Crab_src_indep/" # "BLLac_src_dep/"

If the DL3 index files are note present, run the lstchain_create_dl3_index_files for the given DL3 files

In [ ]:
!lstchain_create_dl3_index_files -d $dir_path --overwrite

In [ ]:
total_datastore = DataStore.from_dir(dir_path)

plot_path = Path(dir_path + 'plots/')
ogip_path = Path(dir_path + 'OGIP/')

plot_path.mkdir(exist_ok=True)
ogip_path.mkdir(exist_ok=True)

# 2. Selection filters for the observations

In [ ]:
# Get the object name from the OBS Table, assuming all the DL3 files are of the same single source.
# If not, then select a single object, to produce the relevant Spectrum Dataset file

obj_name = np.unique(total_datastore.obs_table["OBJECT"])[0]
print("The source is", obj_name)

max_zen = 30 # in deg for a maximum limit on zenith pointing of observations
min_time = 300 # in seconds for minimum livetime of each observation

In [ ]:
total_obs_list = total_datastore.obs_table["OBS_ID"].data
observations_total = total_datastore.get_observations(total_obs_list)

In [ ]:
# If you want to see the full Obs table, run this cell
total_datastore.obs_table

# 3. Make selection of observations

In [ ]:
d_wob = [total_datastore.obs_table["OBS_MODE"]=='WOBBLE']

d_time = [total_datastore.obs_table["LIVETIME"]>min_time]
d_zen = [total_datastore.obs_table["ZEN_PNT"]<max_zen]
d_obj = [total_datastore.obs_table["OBJECT"]==obj_name]

wob_obs_table = total_datastore.obs_table[d_wob[0]*d_zen[0]*d_obj[0]*d_time[0]]
wob_obs_list = total_datastore.obs_table[d_wob[0]*d_zen[0]*d_obj[0]*d_time[0]]["OBS_ID"]

observations_wob = total_datastore.get_observations(wob_obs_list.data)

In [ ]:
print('Wobble observation runs selected are:', wob_obs_list.data)
print(f'Total livetime of all observations: {total_datastore.obs_table["LIVETIME"].data.sum()/3600:.3f} hrs')
print(f'Total livetime of all selected wobble observations {wob_obs_table["LIVETIME"].data.sum()/3600:.3f} hrs')

# 4. Some small functions to get parameters for constructing the Geom for SpectrumDataset

In [ ]:
# Take it as an exercise?
def get_theta_cut_for_geom(dir_path):
    filename_list = []
    for file in os.listdir(dir_path):
        if file.startswith('dl3'):
            filename_list.append(file)
    filename_list = np.sort(filename_list)
    
    theta_cut = u.Quantity(
        Table.read(
            dir_path+filename_list[0], 
            hdu="EFFECTIVE AREA"
        ).meta["RAD_MAX"]
    )
    
    return theta_cut, filename_list


In [ ]:
total_datastore.hdu_table

# 5. Define Target position and energy ranges for reconstructed events

In [ ]:
target_position = SkyCoord.from_name(obj_name, frame='icrs')

In [ ]:
# Long way to find the theta cut used for creating the IRFs

# Select the HDU entry of the first selected wobble observation
hdu_t = total_datastore.hdu_table
hdu_idx = np.where(hdu_t["OBS_ID"] == wob_obs_list[0])[0][0]

hdu_sel = hdu_t[hdu_idx]
print("Base directory of the HDU index file,", hdu_sel.meta["BASE_DIR"])
print("Location of the selected DL3 file, with respect to HDU index file,", hdu_sel["FILE_DIR"])
print("File name of the selected observation,", hdu_sel["FILE_NAME"])

file = Path(hdu_sel.meta["BASE_DIR"]) / hdu_sel["FILE_DIR"] / hdu_sel["FILE_NAME"]

# Checking the fixed global theta cut value, stored as RAD_MAX metadata in all IRF HDUs
theta_cut = Table.read(file, hdu="EFFECTIVE AREA").meta["RAD_MAX"]
print("Theta cut applied for creating the IRF in the selected DL3 file,", theta_cut)

# Converting the string data into astropy.units
on_region_radius = u.Quantity(theta_cut)

In [ ]:
# Provide the minimum, maximum energies in TeV units, and number of bins per decade, to create the 
# required reconstructed and spectral fit energy ranges.

e_reco_min = 0.01 * u.TeV
e_reco_max = 40 * u.TeV

# The following units will be used now to restrict the events in the Spectrum Dataset, only within 
# the energy ranges, in which we want to perform spectral analysis.
e_fit_min = 0.01 * u.TeV
e_fit_max = 40 * u.TeV

# Using bins per decade
e_reco_bin_p_dec = 5

# Calculating the bin size in log scale for the given number of bins per decade
e_reco_bin = int(
    round(
        (np.log10(e_reco_max.value) - np.log10(e_reco_min.value)) * e_reco_bin_p_dec + 1, 0
    )
)

e_reco = MapAxis.from_edges(
    np.logspace(
        np.log10(e_reco_min.value), 
        np.log10(e_reco_max.value), 
        e_reco_bin
    ), 
    unit="TeV", name="energy", interp="log"
)

# 6. Define the base Map geometries for creating the SpectrumDataset

In [ ]:
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

# This will create the base geometry in which to bin the events based on their reconstructed positions
# One can also vary the different parameters here, to get required plots
geom = WcsGeom.create(
    skydir=target_position, npix=(100, 100), 
    binsz=0.05, frame="icrs", axes=[e_reco]
)

In [ ]:
# Exclusion region/source for Crab - RGB J0521+212. 
# Can include specific close gamma-ray objects with respect to the given source, after looking at catalogs
# like http://gamma-sky.net/
RGB_region = CircleSkyRegion(
    center=SkyCoord(183.604, -8.708, unit="deg", frame="galactic"),
    radius=0.5 * u.deg,
)

exclusion_regions = [RGB_region]
exclusion_mask = geom.to_image().region_mask(exclusion_regions, inside=False)

exclusion_mask = WcsNDMap(geom.to_image(), exclusion_mask)
exclusion_mask.plot()
plt.grid()

# 7. Data Reduction chain

In [ ]:
# Create some Dataset and Data Reduction Makers
dataset_empty = SpectrumDataset.create(
    e_reco=e_reco, region=on_region
)
# When not including a PSF IRF, put the containment_correction as False
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, 
    selection=["counts", "exposure", "edisp"]
)

In [ ]:
# The following makers can be tuned and played to check the final Dataset to be used.

# Select the necessary number and size of the OFF regions, to be chosen by this method
bkg_maker = ReflectedRegionsBackgroundMaker(
    exclusion_mask=exclusion_mask,
    min_distance_input=2 * u.rad, # Minimum distance from input region
    max_region_number=10 # Maximum number of OFF regions
) 
# Can also include other parameters, by checking the documentation,
# https://docs.gammapy.org/0.18.2/api/gammapy.makers.ReflectedRegionsBackgroundMaker.html#gammapy.makers.ReflectedRegionsBackgroundMaker

In [ ]:
# Maker for safe energy range for the events.
safe_mask_masker = SafeMaskMaker(
    methods=["aeff-max"], 
    aeff_percent=10
)
# For other arguments and options, check the documentation,
# https://docs.gammapy.org/0.18.2/api/gammapy.makers.SafeMaskMaker.html#gammapy.makers.SafeMaskMaker

# 8. Generate the Spectrum Dataset for all observations

In [ ]:
%%time
# The final object will be stored as a Datasets object
datasets = Datasets()

for obs_id, observation in zip(wob_obs_list, observations_wob):
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(obs_id)), 
        observation
    )
    print('obs_id:', obs_id)
    
    dataset_on_off = bkg_maker.run(
        dataset=dataset, 
        observation=observation
    )
    
    # Some energy masks based on maximum reconstructed energy or spectral fit energy range
    # This maybe ignored if the events with full energy range are required.
    mask_fit = Map.from_geom(
        geom = dataset_on_off.counts.geom, 
        data = dataset_on_off.counts.geom.get_coord()["energy"] < e_reco_max
    )
    dataset_on_off.counts.geom.energy_mask(
        energy_min=e_fit_min, 
        energy_max=e_fit_max
    )
    dataset_on_off.mask_fit = mask_fit
    
    # Add the name of the observed source
    dataset_on_off.meta_table["SOURCE"]=obj_name
    
    # Check the LC and SEDs by applying the safe mask to see the distinction.
    #dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    
    datasets.append(dataset_on_off)    

In [ ]:
print(datasets[0])

# 9. Some plots with the given Dataset

In [ ]:
# Check the OFF regions used for calculation of excess
plt.figure(figsize=(8, 5))
_, ax, _ = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets, legend=True)
plt.grid()

# If need be, redo section 7 and 8, to be sure of the final dataset.
# This could be in the case of using source-dependent dataset

In [ ]:
# For source dependent analysis, check the reconstructed position of all the events, 
# to be sure on the type of dateset we have
for o in observations_wob:
    table=o.events.table
    plt.plot((table["RA"]*24/360),(table["DEC"]), '.')
plt.grid()
plt.gca().invert_xaxis()
plt.xlabel("RA (deg)")
plt.ylabel("Dec (deg)")
#plt.xlim(22.13,21.95)

In [ ]:
info_table = datasets.info_table(cumulative=True)
info_table

In [ ]:
# Plot temporal evolution of excess events and significance value
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")), info_table["excess"], marker="o", ls="none"
)
plt.plot(info_table["livetime"].to("h")[-1:1], info_table["excess"][-1:1], 'r')
plt.xlabel("Sqrt Livetime h^(1/2)")
plt.ylabel("Excess")
plt.grid()
plt.title('Excess vs Square root of Livetime')

plt.subplot(122)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)
plt.grid()
plt.xlabel("Sqrt Livetime h^(1/2)")
plt.ylabel("sqrt_ts")
plt.title('Significance vs Square root of Livetime')
plt.subplots_adjust(wspace=0.5)


In [ ]:
%%time
# Plot the counts+excess, exposure and energy migration of each selected dataset
plt.figure(figsize=(21, len(datasets)*5.5))
j=1
hist_kwargs = {"vmin":0, "vmax":1}

for data in datasets:
    plt.subplot(len(datasets), 3, j)
    data.plot_counts()
    data.plot_excess()
    plt.grid(which="both")
    plt.title(f'Run {data.name} Counts and Excess')
    j += 1
    
    plt.subplot(len(datasets), 3, j)
    data.exposure.plot()
    plt.grid(which='both')
    plt.title(f'Run {data.name} Exposure')
    j += 1
    
    plt.subplot(len(datasets), 3, j)
    if data.edisp is not None:
        kernel = data.edisp.get_edisp_kernel()
        kernel.plot_matrix(add_cbar=True, **hist_kwargs)
        plt.title(f'Run {data.name} Energy Dispersion')
    j += 1
plt.subplots_adjust(hspace=0.3)

# 10. Write all datasets into OGIP files

In [ ]:
# Once the latest dependencies are updated, these warnings will go away
for d in datasets:
    d.to_ogip_files(
        outdir=ogip_path, overwrite=True
    )

In [ ]:
# Read the OGIP files to include the source object name in its headers, to be used for further analysis
for obs in wob_obs_list:
    file = ogip_path/f"pha_obs{obs}.fits"
    
    d1 = fits.open(file)
    d1["REGION"].header["OBJECT"]=obj_name
    d1.writeto(file, overwrite=True)